In [150]:
%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import tensorflow as tf
import numpy as np
import utils, math
from persistence_diagram import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [168]:
import tensorflow as tf
import manifolds


class PManifold(tf.keras.layers.Layer):
    '''
        Model definition for the Persistent Manifold Layer
        The input to this layer is a peristence diagram with
        its points embedded in a m-dim Euclidean space
    '''

    def __init__(self, max_num_of_points, man_dim, num_of_hom, K, manifold='poincare'):
        '''
            Initializes layer params, i.e theta's
        '''
        super(PManifold, self).__init__()
        self.K = K
        self.num_of_hom = num_of_hom
        self.max_num_of_points = max_num_of_points
        self.man_dim = man_dim

        if manifold == 'poincare':
            self.manifold = manifolds.Poincare(man_dim=man_dim)
        if manifold == 'euclidean':
            self.manifold = manifolds.Euclidean()
        if manifold == 'lorenz':
            self.manifold = manifolds.Lorenz(man_dim=man_dim)

        self.x_o = tf.random.normal(shape=(self.man_dim,))  # the fixed point on the manifold
        self.x_o = self.manifold.project_to_manifold(self.x_o)

        theta_init = tf.random_uniform_initializer()
        self.theta = tf.Variable(name='theta',
                                 initial_value=theta_init(shape=(self.num_of_hom,
                                                                 self.K, self.man_dim),
                                                          dtype=tf.float32),
                                 trainable=True)

    def process_dgm(self, dgm, ind):
        '''
            Compute the representation of a diagram
        '''
        padded_dgm = tf.pad(dgm, paddings=[[0, 0], [0, 0], [0, self.man_dim - 2]])
        man_dgm = self.manifold.parametrization(padded_dgm)

        man_dgm = tf.expand_dims(man_dgm, axis=-2)
        man_dgm = tf.repeat(man_dgm, repeats=self.K, axis=-2)
        
        theta = tf.gather(self.theta, indices=ind, axis=0)
        
        # Add lernable vars
        x = tf.add(man_dgm, theta)

        # Make sure that point still belongs to the manifold
        x = self.manifold.project_to_manifold(x)
        
        # Transfer to tangent space
        tangent_x = self.manifold.log_map_x(self.x_o, x)
        reshaped_tangent_x = tf.reshape(tangent_x,
                                        shape=[-1, self.max_num_of_points,
                                               self.K, self.man_dim])
        # Sum out diagram points
        sums = tf.reduce_sum(tangent_x, axis=1)

        # Transform back to manifold
        x_dgm = self.manifold.exp_map_x(self.x_o, sums)

        # Transform to eucledian
        y_dgm = self.manifold.chart(x_dgm)

        return tf.reshape(y_dgm, shape=[-1, self.K, self.man_dim])

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'projection_bases': self.K,
            'num_of_hom': self.num_of_hom,
            'max_num_of_points': self.max_num_of_points,
            'man_dim': self.man_dim,
            'manifold': self.manifold,
            'x_0': self.x_o,
            'theta': self.theta
        })
        return config

    def call(self, inputs):
        '''
            Call method of Keras Layers
        '''
        # Get the diagrams for the two homology classes
        # Two classes are sufficient for images/graphs
        # TODO generalize to m classes in the future
        dgm_0 = inputs[:, 0, :, :]  # zero-th homology class
        dgm_1 = inputs[:, 1, :, :]  # first homology class

        # Get and concat outputs
        out_0 = self.process_dgm(dgm_0, 0)
        out_1 = self.process_dgm(dgm_1, 1)
        out_0 = tf.expand_dims(out_0, axis=1)
        out_1 = tf.expand_dims(out_1, axis=1)
        out = tf.concat([out_0, out_1], axis=1)

        return out


In [209]:
def get_data_images(images_id):
    '''
        Obtains train/test data for the given image set using the provided filtration paramers
    '''

    # Load data
    if images_id == 'fashion_mnist':
        train_images, train_labels, test_images, test_labels = utils.get_mnist_data(fashion=True)
    elif images_id == 'cifar10':
        train_images, train_labels, test_images, test_labels = utils.get_cifar()
    elif images_id == 'mpeg7':
        train_images, train_labels, test_images, test_labels = utils.get_mpeg_data(new_size=32)
    else:  # Load mnist by default
        train_images, train_labels, test_images, test_labels = utils.get_mnist_data()

    ## Set the params of the filtrations
    # Height filtration
    num_of_vects = 30
    angles = np.linspace(0, math.pi / 2, num_of_vects)
    directions = [[round(math.cos(theta), 3), round(math.sin(theta), 3)] for theta in angles]
    directions = np.array(directions)

    # Radial filtration
    center = np.array([[10, 10], [10, 20], [15, 15], [20, 10], [20, 20]])
    radius = np.array([5, 8, 10, 12, 15])
    center = np.array([])
    radius = np.array([])

    # Erosion filtration
    n_iter_er = np.array([1, 2, 3, 50])
    n_iter_er = np.array([])

    # Dilation filtration
    n_iter_dil = np.array([1, 3, 5, 10, 50])
    n_iter_dil = np.array([])

    # Set filtration params
    params = {'cubical': None,
              'height': directions,
              'radial': {'center': center,
                         'radius': radius
                         },
              'erosion': n_iter_er,
              'dilation': n_iter_dil
              }

    # Concat train/test
    N_train = train_images.shape[0]
    images = np.concatenate([train_images, test_images], axis=0)

    # Get PDs for all
    image_pd = ImagePDiagram(images, fil_parms=params, images_id=images_id)
    diagrams = image_pd.get_pds()

    # Split them
    x_train = []
    x_test = []
    for diagram in diagrams:
        x_train.append(diagram[:N_train])
        x_test.append(diagram[N_train:])

    y_train = train_labels
    y_test = test_labels

    return x_train, y_train, x_test, y_test

In [239]:
data_id = 'mpeg7'
x_train, y_train, x_test, y_test = get_data_images(data_id)

Loaded persistence diagrams.


In [211]:
x_train[3].shape

(60000, 2, 20, 2)

In [251]:
man_dim = 2
K = 10
manifold = 'poincare'
units = [256, 128, 70]

num_of_filtrations = len(x_train)
num_of_hom = 2  # = x_train[0].shape[1]
max_num_of_points = []
for i in range(num_of_filtrations):
    max_num_of_points.append(x_train[i].shape[2])
input_shape = [num_of_filtrations, num_of_hom, max_num_of_points]

# Get input shapes
num_of_fil = input_shape[0]
num_of_hom = input_shape[1]
max_num_of_points = input_shape[2]

# Setup an input for each filtration
in_layer = []
inputs = []
for i in range(num_of_fil):
        layer_input_shape = [num_of_hom, max_num_of_points[i], 2]
        cur_input = tf.keras.Input(shape=layer_input_shape)

        # Create Persistent Manifold Layers
        pm_layer = PManifold(max_num_of_points=max_num_of_points[i], man_dim=man_dim,
                             num_of_hom=num_of_hom, K=K, manifold=manifold)
        inputs.append(cur_input)
        in_layer.append(pm_layer(cur_input))


# Flatten
in_layer_2 = tf.concat(in_layer, axis=1)

flat = tf.keras.layers.Flatten()(in_layer_2)

# First dense
dense1 = tf.keras.layers.Dense(units[0],
                                    activation='relu')(flat)

# Batch norm
batch_norm = tf.keras.layers.BatchNormalization()(dense1)

# Second dense
dense2 = tf.keras.layers.Dense(units[1],
                                    activation='relu')(batch_norm)

# Dropout
dropout = tf.keras.layers.Dropout(0.2)(dense2)

# Out
out_layer = tf.keras.layers.Dense(units=units[2])(dropout)

model= tf.keras.Model(inputs=[inputs], outputs=out_layer)


In [252]:
model.summary()

Model: "model_53"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_724 (InputLayer)          [(None, 2, 110, 2)]  0                                            
__________________________________________________________________________________________________
input_725 (InputLayer)          [(None, 2, 35, 2)]   0                                            
__________________________________________________________________________________________________
input_726 (InputLayer)          [(None, 2, 43, 2)]   0                                            
__________________________________________________________________________________________________
input_727 (InputLayer)          [(None, 2, 43, 2)]   0                                            
___________________________________________________________________________________________

In [248]:
# Instantiate an optimizer.
optimizer = tf.keras.optimizers.Nadam(learning_rate=1e-3)

# Instantiate a loss function.
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])
model.fit(x_train,y_train,epochs=200,batch_size=32,validation_data=(x_test, y_test))

Epoch 1/200


/media/hdd/panos/anaconda3/envs/tfgpu/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:433: UserWarning:

Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.



KeyboardInterrupt: 

In [183]:
model1 = tf.keras.Sequential()
# model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(32,32,1)))
# model.add(tf.keras.layers.MaxPooling2D((2,2)))
# model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
# model.add(tf.keras.layers.MaxPooling2D((2,2)))
# model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model1.add(tf.keras.layers.Flatten())
model1.add(tf.keras.layers.Dense(128, activation='relu'))
model1.add(tf.keras.layers.Dense(70))


model1.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
history = model1.fit(x_train[0], y_train, epochs=200, 
                    validation_data=(x_test[0], y_test))
model1.summary()


Epoch 1/200
40/40 [==============================] - 0s 3ms/step - loss: 4.2650 - accuracy: 0.0127 - val_loss: 4.2550 - val_accuracy: 0.0071
Epoch 2/200
40/40 [==============================] - 0s 2ms/step - loss: 4.2214 - accuracy: 0.0238 - val_loss: 4.2604 - val_accuracy: 0.0000e+00
Epoch 3/200
40/40 [==============================] - 0s 2ms/step - loss: 4.1907 - accuracy: 0.0270 - val_loss: 4.2678 - val_accuracy: 0.0143
Epoch 4/200
40/40 [==============================] - 0s 2ms/step - loss: 4.1581 - accuracy: 0.0333 - val_loss: 4.2831 - val_accuracy: 0.0000e+00
Epoch 5/200
40/40 [==============================] - 0s 2ms/step - loss: 4.1307 - accuracy: 0.0405 - val_loss: 4.3002 - val_accuracy: 0.0071
Epoch 6/200
40/40 [==============================] - 0s 2ms/step - loss: 4.0935 - accuracy: 0.0421 - val_loss: 4.3128 - val_accuracy: 0.0000e+00
Epoch 7/200
40/40 [==============================] - 0s 2ms/step - loss: 4.0716 - accuracy: 0.0429 - val_loss: 4.3258 - val_accuracy: 0.0000e+

40/40 [==============================] - 0s 2ms/step - loss: 3.4334 - accuracy: 0.1730 - val_loss: 5.0107 - val_accuracy: 0.0000e+00
Epoch 58/200
40/40 [==============================] - 0s 2ms/step - loss: 3.4193 - accuracy: 0.1762 - val_loss: 5.0239 - val_accuracy: 0.0000e+00
Epoch 59/200
40/40 [==============================] - 0s 2ms/step - loss: 3.4272 - accuracy: 0.1722 - val_loss: 5.0403 - val_accuracy: 0.0000e+00
Epoch 60/200
40/40 [==============================] - 0s 2ms/step - loss: 3.4273 - accuracy: 0.1762 - val_loss: 5.0532 - val_accuracy: 0.0000e+00
Epoch 61/200
40/40 [==============================] - 0s 2ms/step - loss: 3.4110 - accuracy: 0.1786 - val_loss: 5.0624 - val_accuracy: 0.0000e+00
Epoch 62/200
40/40 [==============================] - 0s 2ms/step - loss: 3.4090 - accuracy: 0.1786 - val_loss: 5.0935 - val_accuracy: 0.0000e+00
Epoch 63/200
40/40 [==============================] - 0s 2ms/step - loss: 3.4154 - accuracy: 0.1730 - val_loss: 5.1012 - val_accuracy: 0.

40/40 [==============================] - 0s 2ms/step - loss: 3.2183 - accuracy: 0.2302 - val_loss: 5.7201 - val_accuracy: 0.0000e+00
Epoch 114/200
40/40 [==============================] - 0s 2ms/step - loss: 3.2149 - accuracy: 0.2214 - val_loss: 5.7259 - val_accuracy: 0.0000e+00
Epoch 115/200
40/40 [==============================] - 0s 2ms/step - loss: 3.2115 - accuracy: 0.2286 - val_loss: 5.7436 - val_accuracy: 0.0000e+00
Epoch 116/200
40/40 [==============================] - 0s 2ms/step - loss: 3.2019 - accuracy: 0.2278 - val_loss: 5.7524 - val_accuracy: 0.0000e+00
Epoch 117/200
40/40 [==============================] - 0s 2ms/step - loss: 3.2013 - accuracy: 0.2246 - val_loss: 5.7585 - val_accuracy: 0.0000e+00
Epoch 118/200
40/40 [==============================] - 0s 2ms/step - loss: 3.2036 - accuracy: 0.2238 - val_loss: 5.7738 - val_accuracy: 0.0000e+00
Epoch 119/200
40/40 [==============================] - 0s 2ms/step - loss: 3.2120 - accuracy: 0.2246 - val_loss: 5.7806 - val_accura

Epoch 169/200
40/40 [==============================] - 0s 2ms/step - loss: 3.1041 - accuracy: 0.2579 - val_loss: 6.3461 - val_accuracy: 0.0000e+00
Epoch 170/200
40/40 [==============================] - 0s 2ms/step - loss: 3.1005 - accuracy: 0.2603 - val_loss: 6.3461 - val_accuracy: 0.0000e+00
Epoch 171/200
40/40 [==============================] - 0s 2ms/step - loss: 3.0926 - accuracy: 0.2540 - val_loss: 6.3561 - val_accuracy: 0.0000e+00
Epoch 172/200
40/40 [==============================] - 0s 2ms/step - loss: 3.0821 - accuracy: 0.2571 - val_loss: 6.3597 - val_accuracy: 0.0000e+00
Epoch 173/200
40/40 [==============================] - 0s 2ms/step - loss: 3.0970 - accuracy: 0.2595 - val_loss: 6.3836 - val_accuracy: 0.0000e+00
Epoch 174/200
40/40 [==============================] - 0s 2ms/step - loss: 3.0937 - accuracy: 0.2563 - val_loss: 6.3796 - val_accuracy: 0.0000e+00
Epoch 175/200
40/40 [==============================] - 0s 2ms/step - loss: 3.0912 - accuracy: 0.2611 - val_loss: 6.401